In [1]:
!pip install rdkit
!pip install mordred

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 37.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 KB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.0 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176721 sha256=e436131da44413cd666a800a22aa15fec18b57b4317cc56c696742c6d338b503
  Stored in directory: /root/.cache/pip/wheels/05/95/d1/9e913738f0e8362b3676917b953a60afd76d2b0b99ff8a71ec
Successfully built mordred
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


In [2]:
import numpy as np
import pandas as pd
import rdkit
from rdkit import Chem, rdBase, DataStructs
from rdkit.Chem import rdmolfiles, rdmolops, AllChem, Draw, rdDepictor, Descriptors
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem.Fingerprints import FingerprintMols

from IPython.display import SVG
from IPython.display import HTML
from IPython.display import display

from collections import defaultdict
from tqdm import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('./drive/MyDrive/AI/Material Informatics (MI)/Ring Repeating Units/Table of Supporting Information(SMILES)- Cal band gap_(RU1, RU2, RRU).csv')

In [5]:
df.head()

,NO.,Repeating units_RU1,Repeating units_RU2,SMILES_RU1,SMILES_RU2,SMILES_RRU,DFT band gap/eV
0,1,CH2-CO-NH-CS,CO-NH-CS-CH2,*-CC(=O)NC(-*)=S,*-CC(=S)NC(-*)=O,O=C1CC(=S)N1,2.691
1,2,CH2-CS-C6H4-O,CS-C6H4-O-CH2,*-CC(=S)C1=CC=C(O-*)C=C1,*-COC1=CC=C(C=C1)C(-*)=S,S=C1COC2=CC=C1C=C2,2.041
2,3,C6H4-C6H4-C4H2S-CS,C6H4-C4H2S-CS-C6H4,*-C(=S)C1=CC=C(S1)C1=CC=C(C=C1)C1=CC=C(-*)C=C1,*-C1=CC=C(C=C1)C(=S)C1=CC=C(S1)C1=CC=C(-*)C=C1,S=C1C2=CC=C(S2)C2=CC=C(C=C2)C2=CC=C1C=C2,1.807
3,4,CO-O-C4H2S-CS,O-C4H2S-CS-CO,*-C(=O)OC1=CC=C(S1)C(-*)=S,*-OC1=CC=C(S1)C(=S)C(-*)=O,O=C1OC2=CC=C(S2)C1=S,1.918
4,5,C6H4-CS-C4H2S-O,CS-C4H2S-O-C6H4,*-OC1=CC=C(S1)C(=S)C1=CC=C(-*)C=C1,*-C(=S)C1=CC=C(OC2=CC=C(-*)C=C2)S1,S=C1C2=CC=C(OC3=CC=C1C=C3)S2,1.930


In [6]:
df["mol_ru1"] = df["SMILES_RU1"].apply(lambda x:Chem.MolFromSmiles(x))
df["mol_ru2"] = df["SMILES_RU2"].apply(lambda x:Chem.MolFromSmiles(x))
df["mol_rru"] = df["SMILES_RRU"].apply(lambda x:Chem.MolFromSmiles(x))

In [8]:
#RDKit descriptors
def rdkit_descriptor(mols):
  from rdkit.ML.Descriptors import MoleculeDescriptors
  from rdkit.Chem import  Descriptors
  descriptor_names = [name[0] for name in Descriptors.descList]
  descriptor_calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)
  desc = [descriptor_calculator.CalcDescriptors(mol) for mol in mols]
  df_RDKit = pd.DataFrame(desc, columns=descriptor_names)
  return df_RDKit

In [10]:
ru1 = rdkit_descriptor(df["mol_ru1"])
ru1 = ru1.dropna(1)

ru2 = rdkit_descriptor(df["mol_ru2"])
ru2 = ru2.dropna(1)

rru = rdkit_descriptor(df["mol_rru"])
rru = rru.dropna(1)

<ipython-input-10-d78d6b409dd2>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  ru1 = ru1.dropna(1)
<ipython-input-10-d78d6b409dd2>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  ru2 = ru2.dropna(1)
<ipython-input-10-d78d6b409dd2>:8: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  rru = rru.dropna(1)


In [11]:
# A list for the evaluation result
rmse_sum = {}
r2_sum = {}

RU1

In [12]:
from sklearn.model_selection import train_test_split

X = ru1.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [13]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.163e+01, tolerance: 3.363e-02
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [14]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_ru1: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_ru1: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_ru1: 0.8288573836163803
RMSE(Train): 0.5035367488899946

R_square(Test)_ru1: 0.8004870417505399
RMSE(Test): 0.46949390460481766


In [15]:
rmse_sum['RU1'] = [RMSE_train, RMSE_test] 
r2_sum['RU1'] = [R_square_train, R_square_test] 

RU2

In [16]:
from sklearn.model_selection import train_test_split

X = ru2.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [17]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.647e+01, tolerance: 3.363e-02
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [18]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_ru2: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_ru2: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_ru2: 0.8257877985337561
RMSE(Train): 0.5080323554379069

R_square(Test)_ru2: 0.7152400919481616
RMSE(Test): 0.5608979157307402


In [19]:
rmse_sum['RU2'] = [RMSE_train, RMSE_test] 
r2_sum['RU2'] = [R_square_train, R_square_test] 

RRU

In [20]:
from sklearn.model_selection import train_test_split

X = rru.values
y = df["DFT band gap/eV"].values

#Seperating literature data to training and validation data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

In [21]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)

Lasso(alpha=0.1)

In [22]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

R_square_train = r2_score(y_train, lasso.predict(X_train))
RMSE_train = mean_squared_error(y_train, lasso.predict(X_train), squared=False)
R_square_test = r2_score(y_test, lasso.predict(X_test))
RMSE_test = mean_squared_error(y_test, lasso.predict(X_test), squared=False)

print(f'R_square(Train)_rru: {R_square_train}\nRMSE(Train): {RMSE_train}\n')
print(f'R_square(Test)_rru: {R_square_test}\nRMSE(Test): {RMSE_test}')

R_square(Train)_rru: 0.8317765384461274
RMSE(Train): 0.49922390217867874

R_square(Test)_rru: 0.7782566985288707
RMSE(Test): 0.4949594937844856


In [23]:
rmse_sum['RRU'] = [RMSE_train, RMSE_test] 
r2_sum['RRU'] = [R_square_train, R_square_test] 

Summary

In [24]:
summary = pd.DataFrame(r2_sum, index=["train", "test"])
summary.style.set_caption("Table of R Square (RDKit descriptor)")

,RU1,RU2,RRU
train,0.828857,0.825788,0.831777
test,0.800487,0.715240,0.778257


In [25]:
summary = pd.DataFrame(rmse_sum, index=["train", "test"])
summary.style.set_caption("Table of RMSE (RDKit descriptor)")

,RU1,RU2,RRU
train,0.503537,0.508032,0.499224
test,0.469494,0.560898,0.494959
